# 类的高级主题

## 通过子类拓展

In [1]:
class MyList(list):
    def __getitem__(self, item):
        print(f'Indexing {self} at {item}')
        return list.__getitem__(self, item)

In [2]:
l = MyList([1, 2, 3])

In [3]:
l[2]

Indexing [1, 2, 3] at 2


3

In [4]:
l.append(4)
print(l)

[1, 2, 3, 4]


In [71]:
class MySet(list):
    def __init__(self, value=[]):
        list.__init__([])
        # self.append(value)
        self.concat(value)

    def concat(self, value):
        for x in value:
            if x not in self:
                self.append(x)

    def union(self, other):
        rec = MySet(self)
        rec.concat(other)
        return rec

    def intersection(self, other):
        rec = []
        for x in self:
            if x in other:
                rec.append(x)
        return MySet(rec)

    def __and__(self, other):
        return self.union(other)

    def __or__(self, other):
        return self.intersection(other)

    def __repr__(self):
        return f'MySet({list.__repr__(self)})'

In [72]:
s = MySet([1, 2, 2, 3, 3, 4, 4])

In [73]:
print(s)

MySet([1, 2, 3, 4])


In [74]:
z = MySet([4, 5, 6])
print(z)

MySet([4, 5, 6])


In [75]:
id(s), id(z)

(4451734848, 4425683568)

In [68]:
s.intersection(z)

MySet([4])

In [76]:
s & z

MySet([1, 2, 3, 4, 5, 6])

In [77]:
s | z

MySet([4])

内置属性的获取将跳过实例

In [97]:
class C:
    data = 'spam'

    def __getattr__(self, item):
        print(f'getattr({self}, {item})')
        return getattr(self.data, item)

In [98]:
x = C()

In [99]:
print(x)

In [100]:
x.__getattr__('data')

getattr(<__main__.C object at 0x1096dacd0>, data)


AttributeError: 'str' object has no attribute 'data'

In [87]:
x[0]

TypeError: 'C' object is not subscriptable

In [91]:
x.__add__

getattr(<__main__.C object at 0x109603150>, __add__)


<method-wrapper '__add__' of str object at 0x109463e70>

In [129]:
class C2:
    def __getattr__(self, item):
        print(f'{item})')


c2 = C2()
c2.normal = lambda: 99

In [132]:
c2.normal()

99

In [131]:
c2.normal

<function __main__.<lambda>()>

In [126]:
c2.__add__ = lambda y: 99 + y

In [127]:
c2.__add__(10)

109

In [128]:
c2 + 10

TypeError: unsupported operand type(s) for +: 'C2' and 'int'

显式访问能够正确运⾏

In [134]:
class C:
    data = 'spam'

    def __getattr__(self, item):
        print(f'getattr({self}, {item})')
        return getattr(self.data, item)

In [137]:
x = C()
x.__getitem__(1)

getattr(<__main__.C object at 0x1096cb810>, __getitem__)


'p'

In [138]:
x[1]

TypeError: 'C' object is not subscriptable

In [139]:
x + 'egg'

TypeError: unsupported operand type(s) for +: 'C' and 'str'

In [140]:
x.__add__('egg')

getattr(<__main__.C object at 0x1096cb810>, __add__)


'spamegg'

重新定义

In [169]:
class C:
    data = 'spam'

    def __getattr__(self, item):
        print(f'getattr({self}, {item})')
        return getattr(self.data, item)

    def __getitem__(self, item):
        return self.data[item]

    def __add__(self, other):
        return self.data + other

In [170]:
x = C()
x.__getitem__(1)

'p'

In [171]:
x[1]

'p'

In [172]:
x + 'egg'

'spamegg'

In [173]:
x.__add__('egg')

'spamegg'

In [174]:
type(x).__getitem__(x, 1)

'p'

In [175]:
x.upper

getattr(<__main__.C object at 0x1096bd1d0>, upper)


<function str.upper()>

所有的类⾃动都是新式的

In [176]:
class C: ...

In [177]:
a = C()

In [178]:
type(a), a.__class__

(__main__.C, __main__.C)

In [179]:
type(C), C.__class__

(type, type)

In [183]:
type([1, 2, 3]), [1, 2, 3].__class__

(list, list)

In [184]:
type(type)

type

In [185]:
type(list)

type

类型测试的隐含意义

In [186]:
class C: pass


class D: pass

In [187]:
c, d = C(), D()

In [189]:
type(c), type(d)

(__main__.C, __main__.D)

In [190]:
type(c) == type(d)

False

In [191]:
type(C) == type(D)

True

In [192]:
type(C), type(D), type(type)

(type, type, type)

In [193]:
c1, c2 = C(), C()
type(c1) == type(c2)

True

In [194]:
c.__class__, d.__class__

(__main__.C, __main__.D)

In [197]:
isinstance(c, object), isinstance(d, object)

(True, True)

In [198]:
C.__base__

object

type ⾃身派⽣⾃ object，并且object 派⽣⾃ type

In [200]:
isinstance(type, object)

True

In [201]:
isinstance(object, type)

True

In [202]:
type is object

False

# 钻⽯继承树的影响

In [206]:
class A: attr = 1


class B(A): pass


class C(A): attr = 2


class D(B, C): pass

搜索顺序: x -> D -> B -> C -> A
对属性 attr ⽽⾔，这种搜索⽅式将在类C中找到 attr

In [ ]:
x = D()
x.attr

In [214]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

显式冲突解决

In [212]:
class A: attr = 1


class B(A): pass


class C(A): attr = 2


class D(B, C): attr = B.attr  #指定继承

In [213]:
x = D()
x.attr

1

In [ ]:
"""
File mapattrs.py (3.X + 2.X)

Main tool: mapattrs() maps all attributes on or inherited by an 
instance to the instance or class from which they are inherited.  

Assumes dir() gives all attributes of an instance.  To simulate 
inheritance, uses either the class's MRO tuple, which gives the 
search order for new-style classes (and all in 3.X), or a recursive
traversal to infer the DFLR order of classic classes in 2.X.  

Also here: inheritance() gives version-neutral class ordering; 
assorted dictionary tools using 3.X/2.7 comprehensions.
"""

import pprint


def trace(X, label='', end='\n'):
    print(label + pprint.pformat(X) + end)  # print nicely


def filterdictvals(D, V):
    """
    dict D with entries for value V removed.
    filterdictvals(dict(a=1, b=2, c=1), 1) => {'b': 2}
    """
    return {K: V2 for (K, V2) in D.items() if V2 != V}


def invertdict(D):
    """
    dict D with values changed to keys (grouped by values).
    Values must all be hashable to work as dict/set keys.
    invertdict(dict(a=1, b=2, c=1)) => {1: ['a', 'c'], 2: ['b']}
    """

    def keysof(V):
        return sorted(K for K in D.keys() if D[K] == V)

    return {V: keysof(V) for V in set(D.values())}


def dflr(cls):
    """
    Classic depth-first left-to-right order of class tree at cls.
    Cycles not possible: Python disallows on __bases__ changes. 
    """
    here = [cls]
    for sup in cls.__bases__:
        here += dflr(sup)
    return here


def inheritance(instance):
    """
    Inheritance order sequence: new-style (MRO) or classic (DFLR)
    """
    if hasattr(instance.__class__, '__mro__'):
        return (instance,) + instance.__class__.__mro__
    else:
        return [instance] + dflr(instance.__class__)


def mapattrs(instance, withobject=False, bysource=False):
    """
    dict with keys giving all inherited attributes of instance,
    with values giving the object that each is inherited from.
    withobject: False=remove object built-in class attributes.
    bysource:   True=group result by objects instead of attributes.
    Supports classes with slots that preclude __dict__ in instances.
    """
    attr2obj = {}
    inherits = inheritance(instance)
    for attr in dir(instance):
        for obj in inherits:
            if attr in getattr(obj, '__dict__', {}):
                attr2obj[attr] = obj
                break

    if not withobject:
        attr2obj = filterdictvals(attr2obj, object)
    return attr2obj if not bysource else invertdict(attr2obj)

In [218]:
print('New-style classes in 2.X and 3.X')


class A:         attr1 = 1


class B(A):      attr2 = 2


class C(A):      attr1 = 3


class D(B, C):   pass


I = D()
print('Py=>%s' % I.attr1)  # Python's search == ours?
trace(inheritance(I), 'INH\n')  # [Inheritance order]
trace(mapattrs(I), 'ATTRS\n')  # Attrs  => Source
trace(mapattrs(I, bysource=True), 'OBJS\n')  # Source => [Attrs]

New-style classes in 2.X and 3.X
Py=>3
INH
(<__main__.D object at 0x1095aee10>,
 <class '__main__.D'>,
 <class '__main__.B'>,
 <class '__main__.C'>,
 <class '__main__.A'>,
 <class 'object'>)

ATTRS
{'__dict__': <class '__main__.A'>,
 '__doc__': <class '__main__.D'>,
 '__module__': <class '__main__.D'>,
 '__weakref__': <class '__main__.A'>,
 'attr1': <class '__main__.C'>,
 'attr2': <class '__main__.B'>}

OBJS
{<class '__main__.A'>: ['__dict__', '__weakref__'],
 <class '__main__.B'>: ['attr2'],
 <class '__main__.C'>: ['attr1'],
 <class '__main__.D'>: ['__doc__', '__module__']}



In [227]:
def keysof(V):
    return sorted(K for K in D.keys() if D[K] == V)


D = {'a': 1, 'b': 2, 'c': 1}
keysof(1)

['a', 'c']

In [228]:
{v: keysof(v) for v in set(D.values())}

{1: ['a', 'c'], 2: ['b']}

In [229]:
trace(mapattrs(I, withobject=True), 'OBJS\n')  # Source => [Attrs]

OBJS
{'__class__': <class 'object'>,
 '__delattr__': <class 'object'>,
 '__dict__': <class '__main__.A'>,
 '__dir__': <class 'object'>,
 '__doc__': <class '__main__.D'>,
 '__eq__': <class 'object'>,
 '__format__': <class 'object'>,
 '__ge__': <class 'object'>,
 '__getattribute__': <class 'object'>,
 '__getstate__': <class 'object'>,
 '__gt__': <class 'object'>,
 '__hash__': <class 'object'>,
 '__init__': <class 'object'>,
 '__init_subclass__': <class 'object'>,
 '__le__': <class 'object'>,
 '__lt__': <class 'object'>,
 '__module__': <class '__main__.D'>,
 '__ne__': <class 'object'>,
 '__new__': <class 'object'>,
 '__reduce__': <class 'object'>,
 '__reduce_ex__': <class 'object'>,
 '__repr__': <class 'object'>,
 '__setattr__': <class 'object'>,
 '__sizeof__': <class 'object'>,
 '__str__': <class 'object'>,
 '__subclasshook__': <class 'object'>,
 '__weakref__': <class '__main__.A'>,
 'attr1': <class '__main__.C'>,
 'attr2': <class '__main__.B'>}



In [230]:
trace(mapattrs(I, withobject=False), 'OBJS\n')  # Source => [Attrs]

OBJS
{'__dict__': <class '__main__.A'>,
 '__doc__': <class '__main__.D'>,
 '__module__': <class '__main__.D'>,
 '__weakref__': <class '__main__.A'>,
 'attr1': <class '__main__.C'>,
 'attr2': <class '__main__.B'>}



In [231]:
trace(mapattrs(I, withobject=True, bysource=True), 'OBJS\n')  # Source => [Attrs]

OBJS
{<class 'object'>: ['__class__',
                    '__delattr__',
                    '__dir__',
                    '__eq__',
                    '__format__',
                    '__ge__',
                    '__getattribute__',
                    '__getstate__',
                    '__gt__',
                    '__hash__',
                    '__init__',
                    '__init_subclass__',
                    '__le__',
                    '__lt__',
                    '__ne__',
                    '__new__',
                    '__reduce__',
                    '__reduce_ex__',
                    '__repr__',
                    '__setattr__',
                    '__sizeof__',
                    '__str__',
                    '__subclasshook__'],
 <class '__main__.A'>: ['__dict__', '__weakref__'],
 <class '__main__.B'>: ['attr2'],
 <class '__main__.C'>: ['attr1'],
 <class '__main__.D'>: ['__doc__', '__module__']}



# slot

In [235]:
class A(object): __slots__ = ['a', 'b']; x = 1; y = 2


class B(A):      __slots__ = ['b', 'c']


class C(A):      x = 2


class D(B, C):
    z = 3

    def __init__(self): self.name = 'Bob';


I = D()
trace(mapattrs(I, bysource=True))  # Also: trace(mapattrs(I))


{<__main__.D object at 0x109613c50>: ['name'],
 <class '__main__.C'>: ['x'],
 <class '__main__.D'>: ['__dict__',
                        '__doc__',
                        '__init__',
                        '__module__',
                        '__weakref__',
                        'z'],
 <class '__main__.A'>: ['a', 'y'],
 <class '__main__.B'>: ['__slots__', 'b', 'c']}



In [233]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

# Slot

In [236]:
class A:
    __slots__ = ['age', 'name']

In [237]:
x = A()
x.gender = 'male'

AttributeError: 'A' object has no attribute 'gender'

In [238]:
x.age = 10

In [247]:
x.__dict__

AttributeError: 'A' object has no attribute '__dict__'

In [249]:
x.age = 20

In [250]:
getattr(x, 'age')

20

In [251]:
setattr(x, 'age', 30)

In [252]:
x.age

30

In [240]:
class B:
    job = 'boss'
    __slots__ = ['age', 'name']

In [244]:
b = B()
b.job = 1

AttributeError: 'B' object attribute 'job' is read-only

In [245]:
class C:
    job = 'boss'

In [246]:
c = C()
c.job = 'mgr'

如果没有⼀个属性命名空间字典，你将不能给不在slot 列表中的实例赋值新的名称：

In [253]:
class D:
    __slots__ = ['age', 'name']

    def __init__(self):
        self.job = 'boss'

In [254]:
d = D()

AttributeError: 'D' object has no attribute 'job'

添加__dict__

In [270]:
class E:
    __slots__ = ['age', 'name', '__dict__']
    salary = 100

    def __init__(self):
        self.job = 'boss'

In [271]:
e = E()

In [272]:
e.salary

100

In [273]:
e.pay = 10

In [274]:
e.salary = 200

In [283]:
e.age = 12

In [284]:
for attr in list(getattr(e, '__dict__', [])) + getattr(e, '__slots__', []):
    print(f'{attr}=>{getattr(e, attr, [])}')

job=>boss
pay=>10
salary=>200
age=>12
name=>[]
__dict__=>{'job': 'boss', 'pay': 10, 'salary': 200}


In [285]:
getattr(e, '__dict__', [])

{'job': 'boss', 'pay': 10, 'salary': 200}

In [286]:
list(getattr(e, '__dict__', []))

['job', 'pay', 'salary']

⼀个名称如果不出现在最底层的_slots_ 列表就不会阻⽌它出现在更⾼层的_slots_中。

In [301]:
class A:
    __slots__ = ['a', 'b']


class B(A):
    __slots__ = ['a', 'c']

In [302]:
x = B()
x.a = 10
x.c = 20

In [303]:
x.__slots__

['a', 'c']

In [304]:
x.b = 1

如果父类中没有slot，子类中的slot就没有意义：如果一个子类继承自一个没有slots的父类，为父类创建的_dict_实例属性将总是可访问的，这让子类中的slots变得基本上没有意义。子类仍旧管理它的slot，但却不能通过任何方法计算

In [305]:
class A:
    pass


class B(A):
    __slots__ = ['a', 'c']

In [307]:
x = B()
x.a = 10
x.name = 'bob'

In [308]:
x.__dict__

{'name': 'bob'}

In [309]:
class A:
    __slots__ = ['a', 'c']


class B(A):
    __slots__ = ['a', 'c']

In [310]:
c = B()

In [312]:
c.a = 10
c.c = 1

In [313]:
c.__dict__

AttributeError: 'B' object has no attribute '__dict__'

In [316]:
B.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ['a', 'c'],
              'a': <member 'a' of 'B' objects>,
              'c': <member 'c' of 'B' objects>,
              '__doc__': None})

# Property
不同 之处在于，slot 管理实例的存储，⽽ property 则能任意地拦截访问并计算值。

In [324]:
class properties:
    def getage(self):
        return 40

    def setage(self, value):
        print('set age: %s' % value)
        self._age = value

    age = property(getage, setage, None, None)

In [325]:
x = properties()
x.age

40

In [326]:
x.age = 30

set age: 30


In [355]:
class C:
    __slots__ = ['age']
    data = 'spam'

    def __getattr__(self, item):
        print(f'getattr: {item}')
        if hasattr(self, '__dict__') and item in self.__dict__:
            return self.__dict__[item]
        elif hasattr(self, '__slots__') and item in self.__slots__:
            return self.__slots__[item]
        else:
            return 'not found'

In [357]:
a = C()
a.hello  # 无限递归

getattr: hello
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __dict__
getattr: __di

RecursionError: maximum recursion depth exceeded while calling a Python object

In [358]:
a.data

'spam'

__get__

In [365]:
class MyClass:
    def __get__(self, instance, owner):
        print(f'{instance=}, {owner=}')
        print('get')
        return 42

    def __set__(self, instance, value):
        print('set')


class Person:
    age = MyClass()

In [366]:
a = Person()
a.age

instance=<__main__.Person object at 0x1095fb1d0>, owner=<class '__main__.Person'>
get


42

In [367]:
a.age = 50

set


In [368]:
a.name = 'bob'

# 静态方法

In [396]:
class Spam:
    counter = 0

    def __init__(self):
        Spam.counter += 1

    @staticmethod
    def print_counter():
        print(Spam.counter)


class Other(Spam): ...

In [397]:
c1, c2 = Spam(), Spam()

In [398]:
c2.print_counter()

2


In [399]:
Spam.print_counter()

2


In [400]:
c3 = Spam()
Spam.print_counter()

3


In [401]:
c4 = Other()
Other.print_counter()

4


In [393]:
class Method:
    def normal(self):
        print(f'{self=}, normal')

    @classmethod
    def class_method(cls):
        print(f'{cls=}, class method')

    @staticmethod
    def static_method():
        print('static method')

In [394]:
q = Method()
q.normal()
q.class_method()
q.static_method()

self=<__main__.Method object at 0x109dd51d0>, normal
cls=<class '__main__.Method'>, class method
static method


In [395]:
Method.class_method()

cls=<class '__main__.Method'>, class method


In [407]:
class Spam:
    counter = 0

    def __init__(self):
        Spam.counter += 1

    @classmethod
    def print_counter(cls):
        print(Spam.counter, cls)


class Sub(Spam):
    @classmethod
    def print_counter(cls):
        print('Extra stuff...', cls)
        Spam.print_counter()


class Other(Spam): ...

In [408]:
x = Sub()
y = Other()

In [409]:
x.print_counter()

Extra stuff... <class '__main__.Sub'>
2 <class '__main__.Spam'>


In [410]:
y.print_counter()

2 <class '__main__.Other'>


In [406]:
z = Other()
z.print_counter()

3


使⽤类⽅法统计每个类的实例个数

In [439]:
class Spam:
    counter = 0

    def __init__(self):
        self.count()

    @classmethod
    def count(cls):
        cls.counter = cls.counter + 1
        
    @classmethod
    def print_counter(cls):
        print(cls.counter, cls)


class Sub(Spam):
    counter = 0

    def __init__(self):
        Spam.__init__(self)


class Other(Spam): counter = 0

In [440]:
x = Spam()
a1, a2, a3 = Sub(), Sub(), Sub()
b1, b2 = Other(), Other()

In [437]:
x.print_counter(), a1.print_counter(), b1.print_counter()

1 <class '__main__.Spam'>
3 <class '__main__.Sub'>
2 <class '__main__.Other'>


(None, None, None)